In [20]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import warnings
warnings.filterwarnings('ignore')
import re
import string
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel


In [21]:
df = pd.read_csv('books.csv', error_bad_lines = False)
df.head()

Skipping line 3350: expected 12 fields, saw 13
Skipping line 4704: expected 12 fields, saw 13
Skipping line 5879: expected 12 fields, saw 13
Skipping line 8981: expected 12 fields, saw 13



,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,11/1/2003,Scholastic
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,5/1/2004,Scholastic Inc.
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,0439682584,9780439682589,eng,2690,41428,164,9/13/2004,Scholastic


In [22]:
missing_values = df.isnull().sum()
print(missing_values)

bookID                0
title                 0
authors               0
average_rating        0
isbn                  0
isbn13                0
language_code         0
  num_pages           0
ratings_count         0
text_reviews_count    0
publication_date      0
publisher             0
dtype: int64


In [23]:
content_data = df[['title','authors','average_rating']]
content_data = content_data.astype(str)
content_data['content'] = content_data['title'] + ' ' + content_data['authors'] + ' ' + content_data['average_rating']

In [24]:
content_data = content_data.reset_index()
indices = pd.Series(content_data.index,
index=content_data['title'])

In [25]:
#removing stopwords
tfidf = TfidfVectorizer(stop_words='english')
#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(content_data['authors'])
#Output the shape of tfidf_matrix
tfidf_matrix.shape

(11123, 8448)

In [26]:
cosine_sim_author = linear_kernel(tfidf_matrix, tfidf_matrix)

In [27]:
def get_recommendations_books(title,
cosine_sim=cosine_sim_author):
 idx = indices[title]
 # Get the pairwsie similarity scores of all books with that book
 sim_scores = list(enumerate(cosine_sim_author[idx]))
 # Sort the books based on the similarity scores
 sim_scores = sorted(sim_scores, key=lambda x: x[1],reverse=True)
 # Get the scores of the 10 most similar books
 sim_scores = sim_scores[1:11]
 # Get the book indices
 book_indices = [i[0] for i in sim_scores]
 # Return the top 10 most similar books
 return list(content_data['title'].iloc[book_indices])

In [28]:
def author_book_shows(book):
 for book in book:
  print(book)


In [29]:
books1 = get_recommendations_books('The Hobbit',
cosine_sim_author)
author_book_shows(books1)


The Lord of the Rings (The Lord of the Rings  #1-3)
The Fellowship of the Ring (The Lord of the Rings  #1)
The Lord of the Rings- 3 volumes set (The Lord of the Rings  #1-3)
The Lord of the Rings (The Lord of the Rings  #1-3)
The Hobbit  or There and Back Again
Poems From The Hobbit
The Hobbit: Or There and Back Again
The Hobbit
Tree and Leaf: Includes Mythopoeia and The Homecoming of Beorhtnoth
The Return of the King (The Lord of the Rings  #3)


In [30]:
books2 =get_recommendations_books('The Lost Continent: Travels in Small Town America', cosine_sim_author)
author_book_shows(books2)

Bill Bryson's African Diary
Bryson's Dictionary of Troublesome Words: A Writer's Guide to Getting It Right
In a Sunburned Country
I'm a Stranger Here Myself: Notes on Returning to America After Twenty Years Away
The Lost Continent: Travels in Small Town America
Neither Here nor There: Travels in Europe
Notes from a Small Island
The Mother Tongue: English and How It Got That Way
A Short History of Nearly Everything
A Short History of Nearly Everything (Illustrated Edition)
